In [ ]:
%matplotlib inline
from IPython.display import clear_output
from pytesseract import image_to_string

from srbot import *

In [ ]:
#Tree gnome village agility

def confirm_click(points,order=None):
    if len(points) == 0:
        return False
    if order is None:
        np.random.shuffle(points)
    else:
        points = points[order]
    for i in range(min(10,len(points))):
        move_mouse(*points[i])
        sleep(0.05)
        uptext = get_uptext()
        cyan = find_colors([0,238,238],uptext,tol=0.2)
        if len(cyan) > 100:
            click_mouse(*points[i])
            return True
    return False


target()
total_trips = 0
last_run = mark_time()
logins = 0
while True:
    client = get_client()
    if len(find_bitmap(loginscreen,client)) > 0:
        logins = logins + 1
        if logins > 10:
            raise RuntimeError('Too many logins! Bailing out before we get fishy!')
        login()
        continue
    if mark_time()-last_run > 10*60:
        raise RuntimeError('Took more than 10min to complete course, something is wrong.')
    
    minimap = get_minimap()
    a = find_colors([111,80,43],minimap,tol=(0.1,0.1,0.1),mode='hsl')
    b = find_colors([124,143,184],minimap,tol=(0.1,0.1,0.1),mode='hsl')
    a = filter_far(a,b,20)
    floors = a-[mmxc-mmxs,mmyc-mmys] if len(a) else a
    clusters,counts = cluster(floors,10)
    clusters = clusters[np.argsort(-counts)]
    if len(counts) == 1: #middle platform
        print('climb limb')
        mainscreen = get_mainscreen()
        limb = find_colors([111,85,26],mainscreen,tol=0.045,mode='hsl')
        floor = find_colors([80,56,42],mainscreen,tol=0.045,mode='hsl')
        climbp = filter_near(limb,np.mean(floor.T,axis=1),45)
        if confirm_click(climbp+[msxs,msys]):
            sleep(3.5)
    elif len(counts) > 1:
        if np.random.random() < 0.05:
            polish_minimap()
            run_on(restore_tab='stats')
        centers = np.asarray([np.mean(cluster.T,axis=1) for cluster in clusters])
        dists = np.sqrt(np.sum(np.square(centers),axis=1))
        print(dists)
        if np.min(dists) > 50: #just climbed down
            print('walking to rope net two')
            click_mouse(mmxc,mmyc-15)
            flag_wait(init=0.5,post=0.7)
        else:
            near = np.argmin(dists)
            print('nearest:',near)
            if near == 0: #at log ropes
                if centers[near,1] < 0:
                    print('walk to rope net')
                    click_mouse(*(centers[2]+[mmxc,mmyc-15]))
                    flag_wait(init=0.5,post=0.7)
                else:
                    print('do log balance')
                    mainscreen = get_mainscreen()
                    log = find_colors([99,59,8],mainscreen,mode='hsl',tol=0.1)
                    clusters,counts = cluster(log)
                    log = clusters[np.argmax(counts)]
                    grass = find_colors([139,145,35],mainscreen,mode='hsl',tol=0.1)
                    dirt = find_colors([102,81,35],mainscreen,mode='hsl',tol=0.1)
                    balance = filter_near(filter_near(log,grass,20),dirt,20)
                    if confirm_click(balance+[msxs,msys]):
                        sleep(5.0)
            elif near == 1: #going to pipes
                if centers[near,1] > 0:
                    print('walk to log balance')
                    click_mouse(*(centers[0]+[mmxc,mmyc-15]))
                    flag_wait(init=0.5,post=0.7)
                else:
                    if np.min(dists) > 35: 
                        print('climb rope net two')
                        mainscreen = get_mainscreen()
                        net = find_colors([120,96,70],mainscreen,mode='hsl',tol=0.05)
                        border = find_colors([111,68,13],mainscreen,mode='hsl',tol=0.05)
                        clusters,counts = cluster(border,10)
                        border = clusters[np.argmax(counts)]
                        climbp = filter_near(net,border,10)
                        if confirm_click(climbp+[msxs,msys]):
                            flag_wait(init=0.5,post=0.7)
                            sleep(2.0)
                    else:
                        print('go through pipes')
                        last_run = mark_time()
                        mainscreen = get_mainscreen()
                        pipes = find_colors([122,113,113],mainscreen,mode='hsl',tol=0.05)
                        clusters,counts = cluster(pipes)
                        pipes = np.concatenate(clusters[counts > 50])
                        if confirm_click(pipes+[msxs,msys]):
                            sleep(12.0)
                            clear_output()
                            total_trips += 1
                            print('completed trip %i'%total_trips)
            elif near == 2: #going to climb up
                print('climb rope net')
                mainscreen = get_mainscreen()
                net = find_colors([120,96,70],mainscreen,mode='hsl',tol=0.05)
                border = find_colors([111,68,13],mainscreen,mode='hsl',tol=0.05)
                clusters,counts = cluster(border,10)
                border = clusters[np.argmax(counts)]
                climbp = filter_near(net,border,10)
                if confirm_click(climbp+[msxs,msys]):
                    flag_wait(init=0.5,post=0.7)
                    sleep(2.0)
    else:
        walls = find_colors([238,238,238],minimap,tol=0.1)-[mmxc-mmxs,mmyc-mmys]
        delta = np.mean(walls[:,0])
        if delta > 0:
            print('top left')
            mainscreen = get_mainscreen()
            grass = find_colors([107,118,28],mainscreen,mode='hsl',tol=0.01)
            rope = find_colors([166,134,93],mainscreen,mode='hsl',tol=0.05)
            walk = filter_near(rope,grass,4)+[msxs,msys]
            dist = np.sum(np.square(walk-[msxc,msyc]),axis=1)
            if confirm_click(walk,order=np.argsort(dist)):
                flag_wait(init=0.5,post=0.7)
                sleep(5.0)
        else:
            print('top right')        
            mainscreen = get_mainscreen()
            tree = find_colors([103,78,26],mainscreen,mode='hsl',tol=0.03)
            knot = find_colors([131,105,73],mainscreen,mode='hsl',tol=0.03)
            clusters,counts = cluster(knot,radius=1)
            knot = clusters[np.argmax(counts)]
            climbp = filter_near(tree,knot,35)
            if confirm_click(climbp+[msxs,msys]):
                flag_wait(init=0.5,post=0.7)
                sleep(2.0)